In [ ]:
# @title Import Python Dependencies
%%capture
from IPython import get_ipython
from IPython.display import display
import pandas as pd
import geopandas as gpd
import requests
from io import StringIO
import numpy as np
from scipy.spatial import cKDTree
import urllib.parse
import json
import folium
import branca.colormap as cm
%load_ext rpy2.ipython
!pip install streamlit_folium
!pip install streamlit
import streamlit as st
import plotly.express as px
from streamlit_folium import st_folium
import os

In [ ]:
# @title Import R Dependencies

%%R
suppressMessages(install.packages("tidyverse", quiet = TRUE))
suppressMessages(install.packages("janitor", quiet = TRUE))
suppressMessages(library(tidyverse, quietly = TRUE, warn.conflicts = FALSE))
suppressMessages(library(janitor, quietly = TRUE, warn.conflicts = FALSE))


In [ ]:
# @title Use R to Fetch Election Results
#  @markdown This cell uses R to fetch election results from NYCVote

%%R
suppressMessages({
  # Define file URLs
  results_url_2016 <- "https://vote.nyc/sites/default/files/pdf/election_results/2016/20161108General%20Election/00000100000Citywide%20President%20Vice%20President%20Citywide%20EDLevel.csv"
  results_url_2020 <- "https://vote.nyc/sites/default/files/pdf/election_results/2020/20201103General%20Election/00000100000Citywide%20President%20Vice%20President%20Citywide%20EDLevel.csv"
  results_url_2024 <- "https://vote.nyc/sites/default/files/pdf/election_results/2024/20241105General%20Election/00000100000Citywide%20President%20Vice%20President%20Citywide%20EDLevel.csv"

  # Define local file paths
  results_file_2016 <- "/content/results_2016.csv"
  results_file_2020 <- "/content/results_2020.csv"
  results_file_2024 <- "/content/results_2024.csv"

  # Download files silently
  invisible(download.file(results_url_2016, results_file_2016, quiet = TRUE))
  invisible(download.file(results_url_2020, results_file_2020, quiet = TRUE))
  invisible(download.file(results_url_2024, results_file_2024, quiet = TRUE))

  # Read and clean 2016 results
  election_results_2016 <- suppressMessages(read_csv(results_file_2016, guess_max = 10e3)) %>%
    janitor::clean_names() %>%
    mutate(
      elect_dist = as.numeric(paste0(ad, ed)),
      candidate = str_replace(unit_name, " \\(.+?\\)$", "")
    ) %>%
    group_by(elect_dist) %>%
    summarize(
      republican = sum(tally * grepl("Trump", candidate)),
      democratic = sum(tally * grepl("Clinton", candidate)),
      green = sum(tally * grepl("Stein", candidate)),
      libertarian = sum(tally * grepl("Johnson", candidate)),
      .groups = "drop"
    ) %>%
    mutate(year = 2016)

  # Read and clean 2020 results
  raw_certified_2020_results <- suppressMessages(read_csv(results_file_2020, col_names = FALSE, guess_max = 100e3))
  col_names_2020 <- janitor::make_clean_names(raw_certified_2020_results[1, 1:11])

  election_results_2020 <- raw_certified_2020_results[, 12:22] %>%
    set_names(col_names_2020) %>%
    mutate(
      elect_dist = as.numeric(paste0(ad, ed)),
      candidate = str_replace(unit_name, " \\(.+?\\)$", "")
    ) %>%
    group_by(elect_dist) %>%
    summarize(
      republican = sum(tally * grepl("Trump", candidate)),
      democratic = sum(tally * grepl("Biden", candidate)),
      green = sum(tally * grepl("Hawkins", candidate)),
      libertarian = sum(tally * grepl("Jorgensen", candidate)),
      .groups = "drop"
    ) %>%
    mutate(year = 2020)

  # Read and clean 2024 results
  raw_certified_2024_results <- suppressMessages(read_csv(results_file_2024, col_names = FALSE, guess_max = 100e3))
  col_names_2024 <- janitor::make_clean_names(raw_certified_2024_results[1, 1:11])

  election_results_2024 <- raw_certified_2024_results[, 12:22] %>%
    set_names(col_names_2024) %>%
    mutate(
      elect_dist = as.numeric(paste0(ad, ed)),
      candidate = str_replace(unit_name, " \\(.+?\\)$", "")
    ) %>%
    group_by(elect_dist) %>%
    summarize(
      republican = sum(tally * grepl("Trump", candidate)),
      democratic = sum(tally * grepl("Harris", candidate)),
      .groups = "drop"
    ) %>%
    mutate(year = 2024)

  # Combine datasets and save
  bind_rows(
    election_results_2016,
    election_results_2020,
    election_results_2024
  ) %>%
    mutate(
      green = replace_na(green, 0),
      libertarian = replace_na(libertarian, 0)
    ) %>%
    write_csv("nyc_election_results_by_district.csv")
})


In [ ]:
# @title Create Pivot Table

# Load the data from the specified path, without headers
df = pd.read_csv('/content/results_2024.csv', header=None)

# Pad 'L' (index 11) and 'M' (index 12) columns to make sure they always have the correct number of digits
df[11] = df[11].astype(str).str.zfill(2)  # Padding L (12th column) to 2 digits
df[12] = df[12].astype(str).str.zfill(3)  # Padding M (13th column) to 3 digits

# Concatenate 'L' and 'M' columns to form a 5-digit string
df['Concatenated'] = df[11] + df[12]

# The relevant columns for pivoting should be the vote categories (column U) and tally (column V)
vote_categories = df[20]  # Column U contains the vote categories
tallies = df[21]  # Column V contains the tallies

# Now, let's create a new DataFrame that contains the concatenated value and the respective vote categories with tallies
pivot_data = pd.DataFrame({
    'Concatenated': df['Concatenated'],
    'Vote Category': vote_categories,
    'Tally': tallies
})

# Clean the Tally column by removing commas and converting it to integers
pivot_data['Tally'] = pivot_data['Tally'].replace({',': ''}, regex=True)  # Remove commas
pivot_data['Tally'] = pd.to_numeric(pivot_data['Tally'], errors='coerce')  # Convert to numeric, coercing errors to NaN

# Now, let's pivot the data
pivot_table = pivot_data.pivot_table(index='Concatenated', columns='Vote Category', values='Tally', aggfunc='sum', fill_value=0)

# Convert all values in the pivot table to integers (as they are now numeric)
pivot_table = pivot_table.astype(int)

# Save the pivot table to a CSV file in /content
pivot_table.to_csv('/content/election_results_pivot.csv', index=True, header=True)

# Confirming that the file was saved
print("Pivot table has been saved to /content/election_results_pivot.csv")

Pivot table has been saved to /content/election_results_pivot.csv


In [ ]:
# @title Summarize Election Results

pivot_table = pd.read_csv('/content/election_results_pivot.csv')

# Create election_summary table
election_summary = pd.DataFrame()
election_summary['ed'] = pivot_table['Concatenated']

# Compute new columns
election_summary['TrumpVance'] = (
    pivot_table['Donald J. Trump / JD Vance (Conservative)'] +
    pivot_table['Donald J. Trump / JD Vance (Republican)']
)

election_summary['HarrisWalz'] = (
    pivot_table['Kamala D. Harris / Tim Walz (Democratic)'] +
    pivot_table['Kamala D. Harris / Tim Walz (Working Families)']
)

election_summary['HarrisMinTrump'] = (
    election_summary['HarrisWalz'] - election_summary['TrumpVance']
)

# Fill any remaining NaN values with 0
election_summary.fillna(0, inplace=True)

# Save summary table
election_summary.to_csv('/content/election_summary.csv', index=True, header=True)

# Confirm file save
print("Election summary saved to /content/election_summary.csv")
print(election_summary.shape[0])
print(election_summary.head())


Election summary saved to /content/election_summary.csv
4419
      ed  TrumpVance  HarrisWalz  HarrisMinTrump
0  23001        1011         295            -716
1  23002         957         289            -668
2  23003         330          99            -231
3  23004         871         391            -480
4  23005         936         336            -600


In [ ]:
# @title Fetch Election Districts

# Define the ArcGIS Feature Service URL
url = "https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/arcgis/rest/services/NYC_Election_Districts/FeatureServer/0/query"

# Parameters for querying the data in chunks (pagination)
params = {
    "where": "1=1",  # Fetch all records
    "outFields": "*",  # Retrieve all fields
    "outSR": "4326",  # Output Spatial Reference (WGS 84)
    "f": "geojson",  # Output format as GeoJSON
    "resultOffset": 0,  # Start offset for pagination
    "resultRecordCount": 1000  # Number of records per request (max usually 1000)
}

all_features = []

# Fetch all records using pagination
print("Fetching election district data...")
while True:
    response = requests.get(url, params=params)
    if response.status_code != 200:
        print(f"Failed to fetch data. Status Code: {response.status_code}")
        break

    data = response.json()

    if "features" not in data or not data["features"]:
        break  # Exit loop when no more records are returned

    all_features.extend(data["features"])
    params["resultOffset"] += params["resultRecordCount"]  # Move to next set of records

    print(f"Retrieved {len(all_features)} records...")

# Convert to a GeoDataFrame
if all_features:
    gdf = gpd.GeoDataFrame.from_features(all_features, crs="EPSG:4326")

    # Save to a GeoPackage file
    gdf.to_file("nyed.gpkg", driver="GPKG")
    print(f"Saved full dataset to nyed.gpkg successfully! Total records: {len(gdf)}")
else:
    print("No data retrieved.")


Fetching election district data...
Retrieved 1000 records...
Retrieved 2000 records...
Retrieved 3000 records...
Retrieved 4000 records...
Retrieved 4345 records...
Saved full dataset to nyed.gpkg successfully! Total records: 4345


In [ ]:
# %%
# @title Join Election Results to Districts

# Reset the index to ensure 'Concatenated' is only a column and not an index level
election_summary = election_summary.reset_index(drop=True)

# Load the election districts dataset
nyed = gpd.read_file('/content/nyed.gpkg')

# Check if 'ElectDist' exists in nyed before merging
if 'ElectDist' not in nyed.columns:
    raise KeyError("The column 'ElectDist' does not exist in nyed. Check the column names!")

# Perform the join
nyed = nyed.merge(election_summary, left_on='ElectDist', right_on='ed', how='left') # Perform the merge

# Save the result
output_path = "/content/nyed_votes.gpkg"
nyed.to_file(output_path, driver="GPKG")

# # Print the first few rows to verify the merge
# print(nyed.head())

In [ ]:
# @title Map election results

# Ensure the data is projected correctly
nyed = nyed.to_crs(epsg=4326)

# Get the dataset centroid for proper centering
centroid = nyed.geometry.centroid.unary_union.centroid

# Define a symmetric color scale around zero for HarrisMinTrump
max_abs_value = max(abs(nyed["HarrisMinTrump"].min()), nyed["HarrisMinTrump"].max())
colormap = cm.LinearColormap(
    colors=["red", "white", "blue"],
    vmin=-max_abs_value,
    vmax=max_abs_value,
    caption="Harris Minus Trump Votes"
)

# Create a Folium map
m = folium.Map(
    location=[centroid.y, centroid.x],  # Ensure proper centering
    zoom_start=10,
    tiles="cartodb positron"
)

# Function to style each district based on HarrisMinTrump value
def style_function(feature):
    value = feature["properties"]["HarrisMinTrump"]
    return {
        "fillColor": colormap(value),
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.7,
    }

# Add the GeoJSON layer using the election district boundaries
folium.GeoJson(
    nyed,
    name="Election Districts",
    tooltip=folium.GeoJsonTooltip(fields=["ElectDist", "HarrisMinTrump"],
                                  aliases=["Election District:", "Harris - Trump Votes:"]),
    style_function=style_function
).add_to(m)

# Add the color legend
colormap.add_to(m)

# Display the map
from IPython.display import display
display(m)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# @title Fetch Poll Sites

# URL of the ArcGIS Feature Server layer
url = "https://services6.arcgis.com/EbVsqZ18sv1kVJ3k/ArcGIS/rest/services/NYS_Elections_Districts_and_Polling_Locations/FeatureServer/3/query?where=1%3D1&outFields=*&outSR=4326&f=geojson"

# Load the data into a GeoDataFrame
pollsites = gpd.read_file(url)

# Save to a GeoPackage file in the content directory
output_path = "/content/pollsites.gpkg"
pollsites.to_file(output_path, driver="GPKG")

# Confirm the file has been saved
output_path


'/content/pollsites.gpkg'

In [ ]:
# @title Fetch subway entrances and exits

# API endpoint
base_url = "https://data.ny.gov/resource/i9wp-a4ja.geojson"
limit = 1000  # Maximum records per request
offset = 0  # Start at the first record

# Initialize GeoJSON structure
geojson_data = {
    "type": "FeatureCollection",
    "features": []
}

while True:
    # Construct the API request URL with pagination
    url = f"{base_url}?$limit={limit}&$offset={offset}"

    # Fetch data from the API
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Error: Unable to fetch data (Status Code: {response.status_code})")
        break

    data = response.json()

    # If no more data is returned, stop the loop
    if "features" not in data or not data["features"]:
        break

    # Append features to the GeoJSON structure
    geojson_data["features"].extend(data["features"])

    # Increment offset for pagination
    offset += limit

# Save the GeoJSON file
geojson_output_path = "/content/subwayentranceexit.geojson"
with open(geojson_output_path, "w") as f:
    json.dump(geojson_data, f, indent=4)

# Confirm file save
geojson_output_path


'/content/subwayentranceexit.geojson'

In [ ]:
# @title Fetch Residential Buildings

# Base URL for the MAPPLUTO dataset
base_url = "https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/arcgis/rest/services/MAPPLUTO/FeatureServer/0/query"

# Properly encode the SQL query
where_clause = "LandUse IN ('01','02','03','04')"
encoded_where = urllib.parse.quote(where_clause)

# Construct the full query URL
query_params = f"?where={encoded_where}&outFields=*&outSR=4326&f=geojson"
mappluto_url = base_url + query_params

# Load the data into a GeoDataFrame
mappluto_gdf = gpd.read_file(mappluto_url)

# Save to a local GeoJSON file
output_path = "/content/mappluto_filtered.geojson"
mappluto_gdf.to_file(output_path, driver="GeoJSON")

# Display dataset information
print(f"Filtered data saved to {output_path}")



Filtered data saved to /content/mappluto_filtered.geojson


In [ ]:
# @title Set CRS

subway_entrances = gpd.read_file('/content/subwayentranceexit.geojson')
election_districts = gpd.read_file('/content/nyed_votes.gpkg')

poll_sites = pollsites.to_crs(epsg=4326)
subway_entrances = subway_entrances.to_crs(epsg=4326)
election_districts = election_districts.to_crs(epsg=4326)

In [ ]:
# @title Calculate Distance Between Poll Sites and Subway Entrances

# Extract poll site and subway coordinates
poll_coords = np.array(list(zip(poll_sites.geometry.x, poll_sites.geometry.y)))
subway_coords = np.array(list(zip(subway_entrances.geometry.x, subway_entrances.geometry.y)))

# Use KDTree for fast nearest-neighbor search
subway_tree = cKDTree(subway_coords)
distances, indices = subway_tree.query(poll_coords)

# Add distance to poll_sites DataFrame
poll_sites["nearest_subway_dist"] = distances  # Distance in degrees (~111km per degree)
poll_sites["nearest_subway_id"] = indices  # Index of nearest subway entrance

# Convert degrees to meters (~111,000 meters per degree)
poll_sites["nearest_subway_dist_m"] = poll_sites["nearest_subway_dist"] * 111000

In [ ]:
# @title Calculate Distance From Housing Units to Poll Sites

# Convert to a projected CRS (New York State Plane, EPSG:2263 for NYC)
mappluto_gdf = mappluto_gdf.to_crs(epsg=2263)
poll_sites = poll_sites.to_crs(epsg=2263)

# Compute centroids in projected CRS for accurate distance calculations
mappluto_gdf["centroid"] = mappluto_gdf.geometry.centroid

# Extract centroid coordinates for buildings
building_coords = np.array(list(zip(mappluto_gdf.centroid.x, mappluto_gdf.centroid.y)))

# Extract poll site coordinates
poll_coords = np.array(list(zip(poll_sites.geometry.x, poll_sites.geometry.y)))

# Use KDTree for fast nearest-neighbor search
poll_tree = cKDTree(poll_coords)
distances, indices = poll_tree.query(building_coords)

# Add distance and nearest poll site ID to mappluto_gdf DataFrame
mappluto_gdf["nearest_poll_dist_m"] = distances  # Distance in meters (accurate in projected CRS)
mappluto_gdf["nearest_poll_id"] = indices  # Index of nearest poll site

# Convert back to WGS 84 for export
mappluto_gdf = mappluto_gdf.to_crs(epsg=4326)

# Drop extra geometry column (keeping only the original building geometry)
mappluto_gdf = mappluto_gdf.drop(columns=["centroid"])

# Save updated dataset
output_path = "/content/mappluto_with_poll_distances.geojson"
mappluto_gdf.to_file(output_path, driver="GeoJSON")

# Confirm completion
print(f"Updated MAPPLUTO dataset saved to {output_path}")


Updated MAPPLUTO dataset saved to /content/mappluto_with_poll_distances.geojson


In [ ]:
#  @title Categorize Accessibility

def accessibility_category(distance):
    if distance <= 200:  # 200m is ~2 blocks
        return "Highly Accessible"
    elif distance <= 500:  # 500m is ~5 blocks
        return "Moderately Accessible"
    elif distance <= 1000:  # 1km (~0.6 miles)
        return "Low Accessibility"
    else:
        return "Not Easily Accessible"

poll_sites["accessibility_category"] = poll_sites["nearest_subway_dist_m"].apply(accessibility_category)


In [ ]:
#  @title Aggregate house/poll site accessibility by election district

%%capture
# Load the correct election districts dataset
election_districts = gpd.read_file("/content/nyed.gpkg")

# Ensure CRS matches for spatial operations
election_districts = election_districts.to_crs(epsg=4326)
poll_sites = poll_sites.to_crs(epsg=4326)
subway_entrances = subway_entrances.to_crs(epsg=4326)

# Extract poll site and subway coordinates
poll_coords = np.array(list(zip(poll_sites.geometry.x, poll_sites.geometry.y)))
subway_coords = np.array(list(zip(subway_entrances.geometry.x, subway_entrances.geometry.y)))

# Use KDTree for fast nearest-neighbor search
subway_tree = cKDTree(subway_coords)
distances, indices = subway_tree.query(poll_coords)

# Add distance to poll_sites DataFrame
poll_sites["nearest_subway_dist"] = distances  # Distance in degrees (~111km per degree)
poll_sites["nearest_subway_id"] = indices  # Index of nearest subway entrance

# Convert degrees to meters (~111,000 meters per degree)
poll_sites["nearest_subway_dist_m"] = poll_sites["nearest_subway_dist"] * 111000

# Rename election district column to match if needed
if "ElectDist" in election_districts.columns:
    election_districts.rename(columns={"ElectDist": "district_id"}, inplace=True)

# Ensure district_id exists by checking column names
print("Election Districts Columns:", election_districts.columns)
print("Poll Sites Columns:", poll_sites.columns)

# If district_id is missing in poll_sites, perform spatial join
if "district_id" not in poll_sites.columns:
    poll_sites = gpd.sjoin(poll_sites, election_districts, how="left", predicate="within")

# Aggregate mean subway accessibility by district
district_accessibility = poll_sites.groupby("district_id")["nearest_subway_dist_m"].mean().reset_index()

# Merge aggregated accessibility with election districts
election_districts = election_districts.merge(district_accessibility, on="district_id", how="left")

# Identify districts with missing subway distances
missing_districts = election_districts["nearest_subway_dist_m"].isna().sum()
print(f"Missing districts without poll site accessibility: {missing_districts}")

# If a district has no poll site, assign the nearest poll site's accessibility
if missing_districts > 0:
    # Extract centroids of election districts for spatial lookup
    district_centroids = election_districts.geometry.centroid
    district_coords = np.array(list(zip(district_centroids.x, district_centroids.y)))

    # Extract poll site locations
    poll_coords = np.array(list(zip(poll_sites.geometry.x, poll_sites.geometry.y)))
    poll_distances = poll_sites["nearest_subway_dist_m"].values

    # Build a KDTree for nearest-neighbor search
    poll_tree = cKDTree(poll_coords)

    # Find the nearest poll site for each district centroid
    distances, indices = poll_tree.query(district_coords)

    # Assign the nearest poll site's accessibility to districts without one
    election_districts["nearest_subway_dist_m"].fillna(pd.Series(poll_distances[indices]), inplace=True)

# Save updated dataset
election_districts.to_file('/content/districts_w_dist.gpkg', driver="GPKG")

# Confirm completion
print("Updated election districts with nearest poll site accessibility saved!")


In [ ]:
# @title Identify Election Districts with Poor Accessibility

# Sort election districts by highest average distance to subway
least_accessible = election_districts.sort_values("nearest_subway_dist_m", ascending=False)

# Display top 10 least accessible districts
print(least_accessible[['district_id', 'nearest_subway_dist_m']].head(10))


     district_id  nearest_subway_dist_m
594        33001            8730.433539
185        26014            8730.433539
184        26013            8730.433539
180        26009            8730.433539
596        33003            8438.758571
595        33002            8438.758571
181        26010            7820.826912
178        26007            7820.826912
186        26015            7820.826912
183        26012            7820.826912


In [ ]:
# @title Map accessibility by election district

import folium
import branca.colormap as cm

# Create a folium map centered on NYC
m = folium.Map(location=[40.7128, -74.0060], zoom_start=11, tiles="cartodb positron")

# Define a color scale
colormap = cm.LinearColormap(
    colors=["green", "yellow", "red"],
    vmin=election_districts["nearest_subway_dist_m"].min(),
    vmax=election_districts["nearest_subway_dist_m"].max(),
    caption="Poll Site Distance to Nearest Subway (meters)"
)

# Add election districts as a choropleth layer
folium.GeoJson(
    election_districts,
    name="Poll Site Accessibility",
    tooltip=folium.GeoJsonTooltip(fields=["district_id", "nearest_subway_dist_m"], aliases=["District:", "Avg Distance (m):"]),
    style_function=lambda feature: {
        "fillColor": colormap(feature["properties"]["nearest_subway_dist_m"]) if feature["properties"]["nearest_subway_dist_m"] is not None else "gray",
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.7,
    }
).add_to(m)

# Add legend
colormap.add_to(m)

# Display the map
m


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# @title Language Minorities by Bourough

# API endpoint
base_url = "https://data.cityofnewyork.us/resource/ajin-gkbp.csv"
limit = 1000  # Max rows per request
offset = 0  # Start at first record

# List to store all data
all_data = []

while True:
    # Construct request URL with pagination
    url = f"{base_url}?$limit={limit}&$offset={offset}"

    # Fetch data from API
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Error fetching data (Status Code: {response.status_code})")
        break

    # Convert response to DataFrame
    data = pd.read_csv(StringIO(response.text))

    # If no data is returned, stop the loop
    if data.empty:
        break

    # Append the data
    all_data.append(data)

    # Increment offset for pagination
    offset += limit

# Combine all data into a single DataFrame
if all_data:
    full_data = pd.concat(all_data, ignore_index=True)
    print("Columns in dataset:", full_data.columns)
else:
    print("No data retrieved from API.")

full_data.to_csv('/content/language_data.csv', index=False)


Columns in dataset: Index(['acs_5_year_data_time_period', 'borough', 'borough_cd_code',
       'community_district_name', 'language', 'lep_population_estimate',
       'of_lep_population', 'cvalep_population_estimate',
       'of_cvalep_population'],
      dtype='object')


In [ ]:
# @title Calculate Party Vote Share

import pandas as pd

# Load the pivot table (this was previously saved in your notebook)
pivot_table = pd.read_csv('/content/election_results_pivot.csv')

# Add the column that contains the "ElectDist", which are the election districts
pivot_table.rename(columns={'Concatenated': 'elect_dist'}, inplace=True)

# List of vote categories that we consider relevant for this analysis
vote_categories = [
    'Donald J. Trump / JD Vance (Conservative)',
    'Donald J. Trump / JD Vance (Republican)',
    'Kamala D. Harris / Tim Walz (Democratic)',
    'Kamala D. Harris / Tim Walz (Working Families)',
]

# Create a DataFrame to store the results
party_vote_share = pd.DataFrame()
party_vote_share['elect_dist'] = pivot_table['elect_dist']  # Copy elect_dist

# Compute total votes cast in each district
pivot_table['total_votes'] = pivot_table[vote_categories].sum(axis=1)

# Ensure we don't divide by zero
pivot_table.loc[pivot_table['total_votes'] == 0, 'total_votes'] = 1

# Calculate vote share for each party
for party in vote_categories:
    # Create vote share columns by party, and ensure they are percentages
    party_vote_share[party] = (pivot_table[party] / pivot_table['total_votes']) * 100

# Display the first few rows of the results
print(party_vote_share.head())

# Save the results to a CSV
party_vote_share.to_csv('/content/party_vote_share.csv', index=False)

   elect_dist  Donald J. Trump / JD Vance (Conservative)  \
0       23001                                  11.408882   
1       23002                                   9.711075   
2       23003                                   7.692308   
3       23004                                   4.675119   
4       23005                                   4.716981   

   Donald J. Trump / JD Vance (Republican)  \
0                                66.003063   
1                                67.094703   
2                                69.230769   
3                                64.342314   
4                                68.867925   

   Kamala D. Harris / Tim Walz (Democratic)  \
0                                 21.516080   
1                                 21.508828   
2                                 21.678322   
3                                 29.714739   
4                                 24.764151   

   Kamala D. Harris / Tim Walz (Working Families)  
0                          

2tg0RdDaMPufsyLaCiesZNoCPYW_4Jat3y3bYRpAJkcXNi9Tp

In [1]:
auth_token = '2tg0RdDaMPufsyLaCiesZNoCPYW_4Jat3y3bYRpAJkcXNi9Tp'

In [2]:
%%capture
# Install pyngrok and other dependencies
!pip install -q pyngrok==7.0.0
!pip install streamlit_folium
!pip install streamlit

# Import libraries
import os
import signal
from pyngrok import ngrok
import subprocess
import time

In [3]:
# Define the file path
file_path = '/content/your_app_name.py'

# Check if the file exists and remove it
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"Existing file removed: {file_path}")

# Check for files before the streamlit code is created.
print('Checking for /content/districts_w_dist.gpkg:')
print(os.path.exists('/content/districts_w_dist.gpkg'))
print('Checking for /content/election_summary.csv:')
print(os.path.exists('/content/election_summary.csv'))
print('Checking for /content/pollsites.gpkg:')
print(os.path.exists('/content/pollsites.gpkg'))

# Define the content of the Streamlit app
streamlit_code = """
# import streamlit as st
# import pandas as pd
# import geopandas as gpd
# import folium
# from streamlit_folium import st_folium
# import plotly.express as px
# import os

# print('Streamlit app code starting') #Debug

# # Load data (adjust file paths as needed)
# try:
#   print('Attempting to load data') #Debug
#   # The file we want to use is districts_w_dist.gpkg, since it has the ed column
#   file_path = '/content/districts_w_dist.gpkg'
#   if not os.path.exists(file_path): #Add a check for the existence of the file
#     raise FileNotFoundError(f"'{file_path}' not found.")
#   print('About to read election_districts') #Debug
#   election_districts = gpd.read_file(file_path)
#   print('Finished reading election_districts') #Debug
#   # Rename the column in the loaded data so it matches the merge operation later
#   election_districts.rename(columns={'district_id':'ed'}, inplace=True)
#   print('About to read election_summary') #Debug
#   election_summary = pd.read_csv('/content/election_summary.csv')
#   print('Finished reading election_summary') #Debug
#   print('About to read poll_sites') #Debug
#   poll_sites = gpd.read_file('/content/pollsites.gpkg')
#   print('Finished reading poll_sites') #Debug
# except FileNotFoundError as e:
#   st.error(f"Error loading data: {e}. Please ensure all data files have been generated in previous cells.")

# # Create the title
# st.title('NYC Election Data Dashboard')

# # Add a selection box
# selected_data = st.selectbox(
#   'Select which data you would like to examine:',
#   ('Election Results', 'Poll Site Accessibility')
# )
# if selected_data == 'Election Results':
#   print('Election Results Selected') #Debug
#   # Merge data
#   print('About to merge data') #Debug
#   merged_data = election_districts.merge(election_summary, left_on='ed', right_on='ed', how='left')
#   print('Finished merging data') #Debug

#   # Check if 'HarrisMinTrump' exists in merged_data
#   if 'HarrisMinTrump' not in merged_data.columns:
#       st.error("Error: 'HarrisMinTrump' column is missing in the merged data. Check previous steps for calculation")
#   else:
#       # Show election results
#       st.subheader('Election Results Map')
#       print('Election Results Map Section Started') #Debug

#       # Create a folium map
#       m = folium.Map(location=[40.7128, -74.0060], zoom_start=11, tiles="cartodb positron")

#       # Add the merged data to the map
#       folium.GeoJson(
#           merged_data,
#           name="Election Results",
#           tooltip=folium.GeoJsonTooltip(fields=["ed", "HarrisMinTrump"], aliases=["District:", "Harris - Trump Votes:"]),
#           style_function=lambda feature: {
#               "fillColor": 'blue' if feature["properties"]["HarrisMinTrump"] >= 0 else 'red',
#               "color": "black",
#               "weight": 1,
#               "fillOpacity": 0.7,
#           }
#       ).add_to(m)

#       # Add legend
#       colormap = folium.LinearColormap(
#           colors=["red", "white", "blue"],
#           index=[-merged_data["HarrisMinTrump"].max(), 0, merged_data["HarrisMinTrump"].min()],
#           vmin=-merged_data["HarrisMinTrump"].max(),
#           vmax=merged_data["HarrisMinTrump"].max(),
#           caption="Harris Minus Trump Votes"
#       )
#       colormap.add_to(m)

#       # Display the map
#       st_folium(m, width=700, height=500)

# elif selected_data == 'Poll Site Accessibility':
#   print('Poll Site Accessibility Selected') #Debug
#   # Show poll site accessibility
#   st.subheader('Poll Site Accessibility Map')
#   # Create the choropleth
#   fig = px.choropleth_mapbox(election_districts,
#                                   geojson=election_districts.geometry,
#                                   locations=election_districts.index,
#                                   color='nearest_subway_dist_m',
#                                   mapbox_style="carto-positron",
#                                   zoom=11, center = {"lat": 40.7128, "lon": -74.0060},
#                                   opacity=0.7,
#                                   labels={'nearest_subway_dist_m':'Distance to Subway (m)'})

#   # Set the map layout
#   fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

#   # Show the map in streamlit
#   st.plotly_chart(fig, use_container_width=True)
import streamlit as st
import os
print('Streamlit app code starting')
st.title('Minimal Streamlit App')
print('Streamlit app code finished')
"""

# Write the Streamlit code to the file
with open(file_path, 'w') as f:
    f.write(streamlit_code)

print(f"Streamlit app saved to: {file_path}")

Existing file removed: /content/your_app_name.py
Checking for /content/districts_w_dist.gpkg:
False
Checking for /content/election_summary.csv:
False
Checking for /content/pollsites.gpkg:
False
Streamlit app saved to: /content/your_app_name.py


In [ ]:
# --- Helper Function to Kill Existing Streamlit Processes ---

def kill_all_python_processes():
    """Kills all Python processes."""
    print('kill_all_python_processes has started')
    try:
        for line in os.popen("ps aux | grep python | grep -v grep"):
            fields = line.split()
            pid = fields[1]
            os.kill(int(pid), signal.SIGKILL)
            print(f"Killed Python process with PID: {pid}")
    except Exception as e:
        print(f"Error killing Python processes: {e}")

# --- Streamlit Execution ---

# Kill all Python processes (important!)
kill_all_python_processes()

# Run Streamlit in the background and capture output
print('Attempting to start subprocess')
try:
    process = subprocess.Popen(
        ['streamlit', 'run', '/content/your_app_name.py', '--server.port', '8501'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    print('Subprocess has been started')
except Exception as e:
    print(f"Error while starting Streamlit subprocess: {e}")

# Wait for the process to finish or fail, then get its output
if process.poll() is not None:
    print("Error: Streamlit process has exited prematurely.")
else:
    print('Waiting for process to finish.')
    process.wait()
    print('process has finished')

# Print standard output
print("Streamlit Standard Output:")
for line in process.stdout:
    print(line, end='')
# Print standard error
print("Streamlit Standard Error:")
for line in process.stderr:
    print(line, end='')